In [1]:
import pandas as pd

In [2]:
structure = pd.read_csv("course-217-structure.csv")
data = pd.read_csv("course-217-events.csv")

costs = structure.ix[:, [5, 8]].set_index("step_id").to_dict()["step_cost"]
passed = data[data.action == "passed"]

In [3]:
users_first_time = data.groupby("user_id").apply(lambda x: x.min()).set_index("user_id").to_dict()["time"]

In [4]:
from collections import defaultdict
from tqdm import tqdm_notebook
# текущее кол-во баллов
users = defaultdict(int)

for _, row in tqdm_notebook(passed.sort_values("time").iterrows()):
    users[row.user_id] += costs[row.step_id]

In [5]:
true_users = {k for k, v in users.items() if v >= 24}

passed = passed[passed.user_id.isin(true_users)]

In [6]:
def f(x):
    cur_points = 0
    for _, row in x.iterrows():
        cur_points += costs[row.step_id]
        if cur_points == 24:
            return row.time - users_first_time[row.user_id]

In [7]:
from tqdm import tqdm
tqdm.pandas()

res = passed.sort_values("time").groupby("user_id").progress_apply(f)
res = res[~pd.isnull(res.values)]

100%|██████████| 530/530 [00:07<00:00, 70.60it/s]


In [8]:
",".join(map(str, res.sort_values().head(10).index))

'4313,1264,821,1146,1124,2622,713,972,4280,1291'